# Recap from Lecture 29 on using Pandas

## Import Excel table as Pandas DataFrame object

In [ ]:
import pandas as pd
xls = pd.ExcelFile('Lecture30_data/titanic3.xls')
sheet_1 = xls.parse(0)
sheet_1

## Side-track: Counter class (a subclass of dictionary)

Counter is a subclass of Dictionary class that we are more familiar with. It provides faster way to tally distinct values and saved the results as a dictionary. 

In [ ]:
from collections import Counter
c = Counter('baracadabra')
print(c)
d = c.most_common(3)
print(d)

We want the Counter to return a dictionary subclass with key as the values of "survived" (i.e., either 0 or 1) and values as the number of passengers who have the value.

In [ ]:
counter = Counter(sheet_1.loc[sheet_1["body"].notna(), "survived"].values)
print(counter)

This means that all of the passengers who have non NA 'body' value have 'survived' value equal to 0 (means that they did not survive), which obviously makes sense.

## Count the number of passengers who have non-NaN values in each input variable

In [ ]:
print(sheet_1.shape)
sheet_1.count()

We now want ot obtain all of the columns with no missing information

In [ ]:
criteria = sheet_1.count() == sheet_1.shape[0]
data = sheet_1[criteria.index[criteria]]
data=data.drop(['name','ticket'],axis=1)
data

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data.loc[0:data.shape[0],'sex'] = le.fit_transform(data.sex)
data

## Split data into training and test

In [ ]:
from sklearn import model_selection
X = data.drop(["survived"], axis=1).values
y = data["survived"].values
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y,test_size=0.2, shuffle=True)

## Training classifier 

In [ ]:
from sklearn.linear_model import LogisticRegression
logitreg = LogisticRegression(solver='liblinear')
fit = logitreg.fit(X_train, y_train)

## Get predicted probabilities

First column contains the probabilities for non-survived

Second column contains the probabilities for survived 

In each row, the two probabilities should sum to one

In [ ]:
y_test_prob = fit.predict_proba(X_test)
y_test_prob

To check our predictions, add probabilities for survied (i.e., 2nd column) to the testing data together with the true survived label

In [ ]:
pred = y_test_prob[:,[1]]

true_label = y_test.reshape((y_test.shape[0], 1))

testdata=np.append(X_test, pred, axis=1)

testdata=np.append(testdata, true_label, axis=1)

testdata_df = pd.DataFrame(testdata, columns=['pclass', 'sex', 'sibsp', 'parch', 'pred_prob', 'true_label'])

testdata_df

In [ ]:
print(testdata_df.iloc[0:10].to_latex(index=False))

### Receiver Operator Characteristic (ROC) curve

Often we don't want to set a single threshold but rather evaluate the model based on all thresholds. ROC is the a way to this.

At each threshold, it calculates two rates:

True Positive Rate = True Positives / (True Positives + False Negatives)

False Positive Rate = False Positives / (False Positives + True Negatives)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

# calculate AUC
probs = y_test_prob[:,1]
auc = roc_auc_score(y_test, probs)
print('AUC: %.3f' % auc)

fpr, tpr, thresholds = roc_curve(y_test, probs)

roc_table = np.append(tpr.reshape((tpr.shape[0], 1)),
 fpr.reshape((fpr.shape[0], 1)), axis=1)
roc_table = np.append(roc_table, thresholds.reshape((thresholds.shape[0], 1)), axis=1)

roc_df = pd.DataFrame(roc_table, columns=['TPR', 'FPR', 'Threshold'])

roc_df

In [ ]:
print(roc_df.to_latex(index=False))

In [ ]:
pyplot.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
pyplot.plot(fpr, tpr, marker='.')
pyplot.xlabel("False Positive Rate")
pyplot.ylabel("True Positive Rate")
pyplot.savefig('figures/roc.eps')